# QuiverTools showcase

First, we install QuiverTools.jl in a clean environment.

In [ ]:
using Pkg;
Pkg.activate("QuiverTools-showcase")

In [ ]:
# Pkg.add(url="https://github.com/QuiverTools/QuiverTools.jl")

In [ ]:
using QuiverTools

In [ ]:
Q = mKronecker_quiver(3)

In [ ]:
println("Adjacency matrix: ", Q.adjacency)
println("arrows: ", arrows(Q))
println("Number of vertices: ", nvertices(Q))
println("Number of arrows: ", narrows(Q))

### Basic functionalities of directed graphs are implemented

In [ ]:
is_connected(Q)

In [ ]:
is_acyclic(Q)

On any quiver one can define the bilinear Euler form.

In [ ]:
println("Matrix of the Euler form: ",QuiverTools.Euler_matrix(Q))
Euler_form(Q, [2, 3], [2, 3])

## We construct a QuiverModuli object and analyze some basic properties of it

In [ ]:
M = QuiverModuliSpace(Q, [2, 3])

In [ ]:
is_projective(M)

In [ ]:
is_smooth(M)

In [ ]:
dimension(M)

## We can compute the Hodge polynomial, as well as the Poincaré polynomial of a given quiver moduli:

In [ ]:
H = Hodge_polynomial(M)

In [ ]:
Hodge_diamond(M)

In [ ]:
P = Poincare_polynomial(M)

In [ ]:
Betti_numbers(M) == [1, 0, 1, 0, 3, 0, 3, 0, 3, 0, 1, 0, 1]

In [ ]:
println("Picard rank: ", Picard_rank(M))
println("Index: ", index(M))

## We can study the unstable locus of the parameter space:

In [ ]:
HN = all_HN_types(M, unstable=true)

# Teleman quantization

For each HN type $\mathbf{d}^*$, we compute the upper bound of the Teleman inequality $\eta_{\mathbf{d}^*}$

In [ ]:
η = all_Teleman_bounds(M)

We can compute the weights $W(\mathcal{U}^{\vee}_i \otimes \mathcal{U}_j, \mathbf{d}^*)$ for every HN stratum as well:

In [ ]:
W = all_weights_endomorphisms_universal_bundle(M)

... And we can verify the inequality.

In [ ]:
all(maximum(W[hntype]) < η[hntype] for hntype in HN)

# Semiorthogonal decompositions

## Checking Teleman inequalities for SOD

In [ ]:
Q = mKronecker_quiver(3);
d = [2, 3];
M = QuiverModuliSpace(Q, d, [9, -6]);
HN = all_HN_types(M, unstable=true);
n = dimension(M);

In [ ]:
r = index(M)

In [ ]:
η = all_Teleman_bounds(M);

H = all_weights_irreducible_component_canonical(M);
for hntype in HN
    H[hntype] *= -1
end
H

In [ ]:
chi = [-1, 1];
Ui = all_weights_universal_bundle(M, chi)

In [ ]:
UidualUj = all_weights_endomorphisms_universal_bundle(M)

In [ ]:
println(all(maximum(UidualUj[hntype]) - H[hntype] < η[hntype] for hntype in HN))

println(all(maximum(-Ui[hntype]) - H[hntype] < η[hntype] for hntype in HN))

println(all(-H[hntype] < η[hntype] for hntype in HN))

for twist in 0:r-2
    println(all(maximum(Ui[hntype]) - twist * H[hntype] < η[hntype] for hntype in HN))
 end

... therefore, all the necessary higher cohomologies vanish!

## Checking Euler characteristic vanishing for SOD

The canonical bundle is given by $\sum_{i \in Q_0}(\langle\mathbf{d}, \mathbf{i}\rangle - \langle\mathbf{i}, \mathbf{d}\rangle)\cdot x_{i, 1}$, where the $x_{i, j}$ are the generators of the Chow ring [cf. *Fano quiver moduli*, Franzen--Reineke--Sabatini].

In [ ]:
CH, CHvars = Chow_ring(M)

In [ ]:
QuiverTools.quotient_ideal(CH)

In [ ]:
println("Todd class :", Todd_class(M))
println("Point class: ", point_class(M))

### We can use the methods above to compute the Euler characteristic of a vector bundle, given its Chern character.

The first Chern class of the canonical divisor

In [ ]:
ω = (Euler_form(Q, d, [1, 0]) - Euler_form(Q, [1, 0], d)) * CHvars[1] +
    (Euler_form(Q, d, [0, 1]) - Euler_form(Q, [0, 1], d)) * CHvars[3]

The Chern character of a line bundle $\mathcal{L}$ with $c_1(\mathcal{L}) = a$ is given by $e^a = \sum_{k \in \mathbb{N}} \frac{a^k}{x!}$.

In [ ]:
truncated_exp(x, n) = sum(x^i/factorial(i) for i in 0:n)

Hbundle = truncated_exp(-1//r * ω, n)
Hbundle_dual = truncated_exp(1//r * ω, n)

The bundles $\mathcal{U}_i$ are not of rank 1, so their Chern characters are a bit more complicated.

In [ ]:
x11, x12, x21, x22, x23 = CHvars;

u1     = 2 + x21 + 1//2*x11^2 - x12 + 1//6*x11*x12 - 1//2*x23 - 1//8*x23*x11 + 1//12*x12^2 - 1//80 * x23*x12 - 1//720 * x23^2;
u1star = 2 - x21 + 1//2*x11^2 - x12 - 1//6*x11*x12 + 1//2*x23 - 1//8*x23*x11 + 1//12*x12^2 + 1//80 * x23*x12 - 1//720 * x23^2;

u2     = 3 + x11 + 1//2*x11^2 - x22 - 1//2*x22*x11 + 2//3*x11*x12 + 1//8*x23*x11 + 1//12*x22*x12 - 1//4*x12^2 + 1//120*x23*x12;
u2star = 3 - x11 + 1//2*x11^2 - x22 + 1//2*x22*x11 - 2//3*x11*x12 + 1//8*x23*x11 + 1//12*x22*x12 - 1//4*x12^2 - 1//120*x23*x12;

bundles = [u1, u2];
dual_bundles = [u1star, u2star];

As an example, we can see that the integral of the universal family over the moduli space vanishes:

In [ ]:
integral(M, u1 + u2)

### We compute the Euler characteristics that we need

In [ ]:
m = 3
for s in 1:m-1
    println([integral(M, u*v*Hbundle_dual^s) for u in dual_bundles for v in bundles])
    println([integral(M, u*Hbundle_dual^s) for u in dual_bundles])
    println([integral(M, Hbundle_dual^s)])

end
println([integral(M, u) for u in bundles])
println([integral(M, u*Hbundle_dual) for u in bundles])

... therefore, all the necessary degree-zero cohomologies vanish as well!